Inductive Matrix Completion Based On Graph Neural Networks
- Link: [Paper](https://openreview.net/pdf?id=ByxxgCEYDS)

In [1]:
from pathlib import Path
import sys
sys.path.append('../../')
import math

import pandas as pd
import numpy as np
import torch.nn.functional as F
from torch.optim import Adam
from torch_geometric.loader import DataLoader

from dataset.movielens import MovieLensDataset
from models.igmc import IGMC
from helper.igmc_utils import *

In [2]:
BASE_PATH = Path('../../raw_data/movie-lens/ml-latest-small')
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEED = 42
EPOCHS=2
BATCH_SIZE=512
LR=1e-3
LR_DECAY_STEP = 20
LR_DECAY_VALUE = 10

In [3]:
df_movies = pd.read_csv(BASE_PATH/'movies.csv')
df_links = pd.read_csv(BASE_PATH/'links.csv')
df_tags = pd.read_csv(BASE_PATH/'tags.csv')
df_ratings = pd.read_csv(BASE_PATH/'ratings.csv').drop(labels='timestamp', axis=1)

In [4]:
df_items, genres, genres_mp  = process_movies(df_movies)
(rated_users, rated_users_dict, num_users), (rated_items, rated_items_dict, num_items), ratings = get_nodes(df_ratings)
# print(df_i)
# print(rated_items_dict)
# break
item_features = get_item_features(df_items, rated_items_dict, sparse=False) # dim
# print(item_features)
# break
user_features = get_user_features(df_ratings, df_items, genres, genres_mp, rated_users_dict, n=5, sparse=False) #dim
class_values = np.sort(np.unique(ratings))
rating_dict = {r: i for i, r in enumerate(np.sort(np.unique(ratings)).tolist())}

samples = (rated_users, rated_items, ratings)
user_train_idx, item_train_idx, user_test_idx, item_test_idx, train_labels, test_labels = split(samples, rating_dict)

data = train_labels + 1.
data = data.astype(np.float32)
adj_mat = sp.csr_matrix(
    (data, [user_train_idx, item_train_idx]), 
    shape=[num_users, num_items], 
    dtype=np.float32,
)

train_dataset = MovieLensDataset(
    root='../../raw_data/movie-lens/ml-latest-small',
    adj_mat=adj_mat,
    links=(user_train_idx, item_train_idx),
    labels=train_labels,
    h=1,
    sample_ratio=1,
    max_nodes_per_hop=200,
    u_features=user_features,
    v_features=item_features,
    class_values=class_values,
)

test_dataset = MovieLensDataset(
    root='../../raw_data/movie-lens/ml-latest-small',
    adj_mat=adj_mat,
    links=(user_test_idx, item_test_idx),
    labels=test_labels,
    h=1,
    sample_ratio=1,
    max_nodes_per_hop=200,
    u_features=user_features,
    v_features=item_features,
    class_values=class_values,
)

train_loader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, BATCH_SIZE, shuffle=False, num_workers=0)

In [5]:
for batch in enumerate(train_loader):
    print(batch)
    break

(0, DataBatch(x=[88423, 4], edge_index=[2, 634072], y=[512], edge_type=[634072], u_feature=[512, 20], v_feature=[512, 20], batch=[88423], ptr=[513]))


In [6]:
batch[1].edge_index.shape

torch.Size([2, 634072])

In [7]:
batch[1].edge_type.shape

torch.Size([634072])

In [8]:
batch[1].x.shape

torch.Size([88423, 4])

In [9]:
num_relations = len(class_values)
n_features = user_features.shape[1] + item_features.shape[1]
model = IGMC(
    train_dataset, 
    num_relations=num_relations, 
    num_bases=4,   
    side_features=True, 
    n_side_features=n_features, 
)

In [10]:
model.to(DEVICE)
model.reset_parameters()
optimizer = Adam(model.parameters(), lr=LR, weight_decay=0)

for epoch in range(1, EPOCHS+1):
    model.train()
    train_loss_all = 0
    for idx, train_batch in enumerate(train_loader):
        optimizer.zero_grad()
        train_batch = train_batch.to(DEVICE)
        y_pred = model(train_batch)
        y_true = train_batch.y
        train_loss = F.mse_loss(y_pred, y_true)
        if idx % 20 == 0:
            print(round(train_loss.item(), 2), end=' ')
        train_loss.backward()
        train_loss_all += BATCH_SIZE * float(train_loss)
        optimizer.step()
        torch.cuda.empty_cache()
        
    train_loss_all = train_loss_all / len(train_loader.dataset)
    
    print('epoch', epoch,'; train loss', train_loss_all)

    if epoch % LR_DECAY_STEP == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] / LR_DECAY_VALUE

tensor([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        ...,
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]])
torch.Size([92439, 4])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (369756x3233 and 4x32)

In [ ]:
model.eval()
test_loss = 0
for test_batch in test_loader:
    test_batch = test_batch.to(DEVICE)
    with torch.no_grad():
        y_pred = model(test_batch)
    y_true = test_batch.y
    test_loss += F.mse_loss(y_pred, y_true, reduction='sum')
mse_loss = float(test_loss) / len(test_loader.dataset)

print('test MSE loss', mse_loss)
print('test RMSE loss', math.sqrt(mse_loss))

In [ ]:
y_true

In [ ]:
n_round(y_pred, 0.5)